# Тема 32. Итоговый проект. Игра Codenames. Решение

Ваша задача сделать упрощенную версию игры [Codenames]( https://ru.wikipedia.org/wiki/Codenames), в которой компьютер будет выступать как ведущий, будет придумывать ассоциации к словам.

# Правила игры


На игровом поле напечатано `cnt_rows` (=5) на `cnt_cols` (=5) игровых слов `words`.

Эти игровые слова выбраны случайно в начале игры из всех допустимых слов `codenames`, которые мы загрузим отдельно. 

Допустимые слова `codenames` выбраны из словаря `glove.6B.300d.npy`, в котором каждому слову из `word_list` сопоставлены вектора `vectors`.

Из игровых слов `words` случайно выбираются `cnt_agents` (=8) слов, которые будут задуманы компьютером, назовем их `my_words`.

Каждый раунд компьютер будет выбирать одно или несколько слов из `my_words`, придумывать им одну ассоциацию `clue`.

Выведет на экран подсказку: эту ассоциацию `clue` и число слов из `my_words`, с которыми она связана, например 'car 2' означает, что слово car относится к двум задуманным словам на экране. Также показано сколько задуманных слов еще не разгадано. Разрешается выводить и число - степень "подходящести" ассоциации словам.

Пример:
```
  france    honey      cat     palm    scale 
  spider    ghost  octopus   mexico      --- 
     ---      mug   change   button triangle 
  cotton    hotel      jet     king     pole 
   bugle     suit   turkey     shop    dwarf 

Thinking....................

Clue: "coats 2" (certainty 3.76, remaining words 7)

Your guess: cotton
Correct!
Your guess: suit
Correct!

```


Игрок, видя на экране все слова words и подсказку, может ввести одно из игровых слов `words`. Во вводе принимаются только слова из `words`, показанные на экране.

Если игрок угадал и введенное слово совпало с одним из задуманных `my_words`, то игрок продолжает угадывание (ввод слов). Угаданное слово удаляется из `words` и `my_words`, счетчик неразгаданных слов уменьшается.

Если игрок не угадал, т.е. введенное слово не совпало с одним из задуманных `my_words`, то раунд завершается, компьютер придумывает новую ассоциацию, начинается новый раунд. Введенное слово удаляется из `words` (на экране заменяется на ---). Это ограничит количество попыток угадать слово.

Игра продолжается, пока все слова из `my_words` не будут угаданы.

В этой простой версии невозможно проиграть, поэтому соревнуйтесь кто быстрее, за меньшее число раундов, завершит игру. 

Ваша цель - сделать игру более привлекательной и разумной. Базовый вариант приведен ниже. 

Что можно изменить:
- словарь, там много мусора и дурацких слов.
- векторные представления слов, например, заменить glove на fasttext.
- изменить способ поиска ассоциации.
- запретить в ассоциациях однокоренные слова.
- усложнить правила, например, добавить слово-убийцу, такое, что если его ввести, игра завершается поражением.
- красивости ввода, вывода, добавить графику, все на ваше усмотрение.

Презентуйте вашу игру, обсудите с учителем и товарищами, поиграйте в игры ваших товарищей, оцените достоинства и найдите недостатки. Расскажите, какие темы из изученных в этом курсе пригодились для написания игры.






## Способ поиска ассоциации
Ассоциации `clue` не могут совпадать с `my_words`, не могут повторяться, для чего будем вести черный список запрещенных на данном раунде слов, в который входят все изначальные `my_words` и задуманные на прошлых раундах `clue`.

Как именно искать ассоциации это ваш выбор, важно, чтобы: 
* ассоциация подходила к нескольким задуманным словам
* ассоциация не повторялась и не совпадала с самими задуманными словами
* желательно уметь регулировать количество задуманных слов подходящих к ассоциации.


В базовом примере ассоциации ищутся так:
* Негативные слова – все слова из `words` кроме `my_words`
* Позитивные слова - `my_words`
* 1) Находим попарные скалярные произведения всех векторов `vectors` и векторов негативных слов, для каждого `vectors` сохраняем максимальное по негативным словам скалярное произведение `nm`.
* 2) Находим попарные скалярные произведения всех векторов vectors и векторов позитивных слов `pm`.
* 3) В цикле по количеству `vectors` проверяем:

Если для текущего слова максимальное `pm` меньше чем `nm` или это слово находится в черном списке, то пропускаем такое слово. Отбор по негативным словам нужен, чтобы ассоциации лучше подходили к задуманным словам, а не всем. Нет смысла брать ассоциацию которая сильно связана с не задуманным словом, это обман игрока.

Иначе, сортируем `pm` для данного слова в порядке возрастания, рассчитываем для слова степень подходящести `real_score`, если эта величина лучше, чем была у других слов, то это слово и будет взято в качестве ассоциации.

Хитрость в том, как считать `real_score`. Вам разрешается придумать и свой метод. 

Здесь рассчитывают из следующих соображений:
- скалярное произведение векторов говорит нам, насколько похожи эти вектора и слова им соответствующие.
- находим разницу похожести и нижней границы. Нижняя граница это похожесть негативных слов. Нет смысла отбирать слово, которое хуже чем негативное.
- положительные слова упорядочены по возрастанию похожести. Самое похожее - последнее в списке. Скорей всего график похожести это возрастающая функция (например, степенная). Если бы мы брали только самое похожее слово, то оно всегда было бы одно, и ассоциация подходила бы только к одному слову. Это не хорошо. Поэтому изменим график похожести, так, чтобы для менее похожих положительных слов похожесть была побольше, чем рассчитано изначально. Здесь применяется умножение на $ (n-i)^a $, где n - число слов в `my_words`, i - номер (индекс) положительного слова, a - некоторый параметр, который можно регулировать. Действительно, для последнего, т.е. лучшего, положительного слова множитель получится 1, похожесть не поменяется. Для первого, самого не похожего, похожесть увеличится в $ n^a $ раз и возможно сможет преодолеть порог отбора. Так мы разрешаем отбирать несколько слов из положительных, которым соответствует ассоциация. Важно не перестараться, иначе все слова будут помечены как подходящие, что тоже не хорошо.    




# Базовое решение
Вы можете использовать идеи и части кода базового решения для создания своей игры. 

In [1]:
import random
import re
import numpy as np
import math

from typing import List, Tuple, Iterable

# файл лога, чтобы видеть что компьютер отбирает для ассоциаций
log_file = open("log_file", "w")

In [2]:
# загружаем словарь разрешенных слов codenames 
!wget https://raw.githubusercontent.com/thomasahle/codenames/master/wordlist
!wget https://raw.githubusercontent.com/thomasahle/codenames/master/wordlist2


--2021-08-15 14:23:00--  https://raw.githubusercontent.com/thomasahle/codenames/master/wordlist
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4633 (4.5K) [text/plain]
Saving to: ‘wordlist’

wordlist            100%[===================>]   4.52K  --.-KB/s    in 0s      

2021-08-15 14:23:00 (36.9 MB/s) - ‘wordlist’ saved [4633/4633]

--2021-08-15 14:23:00--  https://raw.githubusercontent.com/thomasahle/codenames/master/wordlist2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2482 (2.4K) [text/plain]
Saving to: ‘wordlist2’

wordl

In [3]:
# загружаем словарь и вектора всех рабочих слов
import gdown 
# обработанные слова
url='https://drive.google.com/uc?id=1wKlw3cTP7OXrjQeyJxAwZHiZW0EVlwQN'
output ='words' # название для загружаемого файла
gdown.download(url, output, quiet=False)

# обработанные вектора 
url='https://drive.google.com/uc?id=12WHGjEU6wbVgDT8BD3k0cUa7kStM8uIQ'
output ='glove.6B.300d.npy' # название для загружаемого файла
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1wKlw3cTP7OXrjQeyJxAwZHiZW0EVlwQN
To: /content/words
3.36MB [00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=12WHGjEU6wbVgDT8BD3k0cUa7kStM8uIQ
To: /content/glove.6B.300d.npy
480MB [00:04, 103MB/s]


'glove.6B.300d.npy'

Класс для ввода\вывода.

In [4]:
class TerminalReader:
    # запрос ввода игрока
    def read_picks(
        self, words: List[str], my_words: Iterable[str], cnt: int
    ) -> List[str]:
        picks = []
        while len(picks) < cnt: # угадать можно столько раз, скольким словам соответствует ассоциация
            guess = None
            while guess not in words: # если введено не разрешенное слово, будем просить ввести разрешенное
                guess = input("Your guess: ").strip().lower() # просим ввести слово
            picks.append(guess)
            if guess in my_words:
                print("Correct!") # угадал - продолжаем раунд
            else:
                print("Wrong :(") # не угадал - выходим к следующем раунду
                break
        return picks
    # печать слов
    def print_words(self, words: List[str], nrows: int):
        longest = max(map(len, words))
        print()
        for row in zip(*(iter(words),) * nrows):
            for word in row:
                print(word.rjust(longest), end=" ")
            print()
        print()
 

Класс для базового варианта игры.

In [5]:
class Codenames:
    # инициализация
    def __init__(self, cnt_rows=5, cnt_cols=5, cnt_agents=8, agg=.6):
        """
        :param cnt_rows: число рядов слов.
        :param cnt_cols: число столбцов слов.
        :param cnt_agents: число задуманных слов.
        :param agg: влияет на размер группы для ассоциации [0, infinity). Больше - более узкие группы.
        """
        self.cnt_rows = cnt_rows
        self.cnt_cols = cnt_cols
        self.cnt_agents = cnt_agents
        self.agg = agg

        # другие параметры
        self.vectors = np.array([]) # вектора всех слов
        self.word_list = [] # список всех слов
        self.weirdness = [] # дополнительная "бесполезность" слов
        self.word_to_index = {} # словарь, для перевода слова в его индекс
        self.codenames = [] # список разрешенных в игре слов, может отличаться от word_list
    # загрузка векторов
    def load(self, datadir):
        # загрузка Glove векторов
        print("...Loading vectors")
        self.vectors = np.load(f"{datadir}/glove.6B.300d.npy")

        
        print("...Loading words")
        # загрузка списка всех слов в glove
        self.word_list = [w.lower().strip() for w in open(f"{datadir}/words")]
        # бесполезность слов
        self.weirdness = [math.log(i + 1) + 1 for i in range(len(self.word_list))]

        # словрь для перевода из слова в индекс
        print("...Making word to index dict")
        self.word_to_index = {w: i for i, w in enumerate(self.word_list)}

        # загрузка разрешенных в игре слов
        print("...Loading codenames")
        self.codenames: List[str] = [
            word
            for word in (w.lower().strip().replace(" ", "-") for w in open("wordlist2"))
            if word in self.word_to_index
        ]

        print("Ready!")
    # перевод из слова в вектор
    def word_to_vector(self, word: str) -> np.ndarray:
        """
        :param word: To be vectorized.
        :return: The vector.
        """
        return self.vectors[self.word_to_index[word]]
    # поиск ассоциации
    def find_clue(
        self, words: List[str], my_words: List[str], black_list: Iterable[str]
    ) -> Tuple[str, float, List[str]]:
        """
        :param words: слова в игре.
        :param my_words: задуманные слова.
        :param black_list: черный список слов.
        :return: (лучшая ассоциация, похожесть, группа слов которой подходит ассоциация)
        """
        print("Thinking", end="", flush=True)

        # негативные слова.
        negs = [w for w in words if w not in my_words]
        # их максимальная похожесть
        nm = (
            self.vectors @ np.array([self.word_to_vector(word) for word in negs]).T
        ).max(axis=1)
        
        # позитивные слова и их похожесть
        pm = self.vectors @ np.array([self.word_to_vector(word) for word in my_words]).T

        best_clue, best_score, best_k, best_g = None, -1, 0, ()
        # цикл перебора всех слов словаря для поиска ассоциации
        for step, (clue, lower_bound, scores) in enumerate(zip(self.word_list, nm, pm)):
            if step % 20000 == 0:
                print(".", end="", flush=True)

            # если похожесть меньше чем у негативных слов - пропускаем
            if max(scores) <= lower_bound or clue in black_list:
                continue

            # сортируем по похожести.
            ss = sorted((s, i) for i, s in enumerate(scores))
            
            # пересчитываем похожесть в "real score" чтобы работать с группой слов 
            real_score, j = max(
                (
                    (s - lower_bound)
                    * ((len(ss) - j) ** self.agg - .99)
                    / self.weirdness[step],
                    j,
                )
                for j, (s, _) in enumerate(ss)
            )

            # если текущая пересчитанная похожесть лучше чем была раньше (для других слов),
            # то отбираем все положительные слова лучшие (т.е. последние) чем найденное (включая его самого)       
            if real_score > best_score:
                group = [my_words[i] for _, i in ss[j:]]
                best_clue, best_score, best_k, best_g = (
                    clue, # лучшая ассоциация
                    real_score, # лучшая похожесть
                    len(group), # количество отобранных положительных слов в группе
                    group, # сама отобранная группа
                )

        # пустая линия.
        print()

        return best_clue, best_score, best_g
    # запуск игры
    def play_spymaster(self, reader: TerminalReader):
        """
        Play a complete game, with the robot being the spymaster.
        """
        # выбираем случайно игровые слова
        words = random.sample(self.codenames, self.cnt_rows * self.cnt_cols)
        # из них выбираем случайно задуманные
        my_words = set(random.sample(words, self.cnt_agents))
        # черный список слов
        used_clues = set(my_words)
        # раунды, пока остались не разгаданные слова
        while my_words:
            # печатаем игровое поле
            reader.print_words(words, nrows=self.cnt_rows)
            # ищем ассоциацию
            clue, score, group = self.find_clue(words, list(my_words), used_clues)
            # сохраняем в лог
            group_scores = np.array(
                [self.word_to_vector(w) for w in group]
            ) @ self.word_to_vector(clue)
            print(clue, group, group_scores, file=log_file, flush=True)
            # добавляем ассоциацию в черный список
            used_clues.add(clue)

            # печатаем подсказку игроку
            print()
            print(
                'Clue: "{} {}" (certainty {:.2f}, remaining words {})'.format(
                    clue, len(group), score, len(my_words)
                )
            )
            print()
            # вызываем запрос ввода и 
            for pick in reader.read_picks(words, my_words, len(group)):
                words[words.index(pick)] = "---" # правильно введенные слова удаляем 
                if pick in my_words:
                    my_words.remove(pick) # угаданные тоже удаляем

 

In [6]:
# создаем игру
cn = Codenames(agg=0.6)
# загружаем словари и вектора
cn.load(".")
# инициализируем класс  ввода\вывода
reader = TerminalReader()
# запускаем игру
cn.play_spymaster(reader,)

...Loading vectors
...Loading words
...Making word to index dict
...Loading codenames
Ready!

  australia      berlin         mug      marble      shadow 
      plate        note       press shakespeare       board 
      novel         lap        calf      lawyer       beach 
      mount        deck        code         car       knife 
    america        life       olive       lemon         cat 

Thinking....................

Clue: "ferrari 2" (certainty 0.87, remaining words 8)

Your guess: car
Correct!
Your guess: board
Correct!

  australia      berlin         mug      marble      shadow 
      plate        note       press shakespeare         --- 
      novel         lap        calf      lawyer       beach 
      mount        deck        code         ---       knife 
    america        life       olive       lemon         cat 

Thinking....................

Clue: "canterbury 2" (certainty 0.47, remaining words 6)

Your guess: berlin
Wrong :(

  australia         ---         mug    

# Ссылки
Использованы и адаптированы материалы:
* https://github.com/thomasahle/codenames 
